In [35]:
from dotenv import load_dotenv
import streamlit as st
import json
import pandas as pd
import redis
import os
import matplotlib.pyplot as plt
import plotly.express as px

In [36]:
st.set_page_config(page_title="What's in my Kitchen?", page_icon="🥘")
st.title("🥘 What's in my Kitchen?")

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [37]:
load_dotenv()
r_host = os.environ.get('RD_HOST')
r_port = os.environ.get('RD_PORT')
r_pass = os.environ.get('RD_PASS')

In [38]:
print(os.getcwd())

d:\Codes\lynx_ai\pages


In [39]:
with open('categories.txt') as c:
    data = c.read()
data = data.replace("'", "\"")
categories = json.loads(data)

In [40]:
categories

{'Bakery': ['durum', 'salt', 'sugar', 'bread'],
 'Canned goods': ['kidney beans', 'mushroom', 'tomato puree'],
 'Dairy': ['butter', 'cheese', 'egg', 'eggs', 'milk', 'yogurt'],
 'Fish': ['salmon', 'tuna'],
 'Fruits': ['apple', 'orange', 'tangerine'],
 'Grains': ['flour', 'musli', 'pasta', 'rice'],
 'Meat': ['beef', 'chicken', 'chicken breast', 'pork'],
 'Oil': ['cooking oil', 'olive oil'],
 'Spices': ['chilli powder',
  'garam masala',
  'garlic paste',
  'garlic powder',
  'ginger paste',
  'turmeric powder'],
 'Vegetables': ['carrot', 'garlic', 'onion', 'potatoes', 'tomato']}

In [41]:
@st.cache_data
def redis_call(host, port, password):

    r = redis.Redis(
        host=host,
        port=port,
        password=password)

    keys = r.keys()
    values = r.mget(keys)

    data = {}

    for key, value in zip(keys, values):
        data[f"{key.decode()}"] = f"{value.decode()}"

    return data

In [42]:
data = redis_call(r_host, r_port, r_pass)

In [43]:
data

{'Chicken Breast ': '{"quantity": "0", "unit": "gram"}',
 'Cooking Oil': '{"quantity": "496", "unit": "mililitre"}',
 'Sugar': '{"quantity": "-1", "unit": "kilogram"}',
 'Ginger Paste': '{"quantity": "498", "unit": "gram"}',
 'Olive Oil': '{"quantity": "498", "unit": "mililitre"}',
 'Turmeric powder': '{"quantity": "99", "unit": "gram"}',
 'Chicken': '{"quantity": "-499", "unit": "kilogram"}',
 'Kidney Beans': '{"quantity": "-99", "unit": "kilogram"}',
 'Flour': '{"quantity": "400", "unit": "gram"}',
 'Rice': '{"quantity": "-1", "unit": "kilogram"}',
 'Musli': '{"quantity": "-49", "unit": "kilogram"}',
 'Tuna': '{"quantity": "150", "unit": "gram"}',
 'Tomato': '{"quantity": "-1", "unit": "kilogram"}',
 'Carrot': '{"quantity": "0", "unit": "kilogram"}',
 'Garlic Paste': '{"quantity": "498", "unit": "gram"}',
 'Onion': '{"quantity": "0", "unit": "kilogram"}',
 'Garlic ': '{"quantity": "250", "unit": "gram"}',
 'Cheese': '{"quantity": "500", "unit": "gram"}',
 'Salt': '{"quantity": "250",

In [44]:
df = pd.DataFrame()

for key, value in data.items():
    if key != 'key':
        temp_df = pd.DataFrame.from_dict(json.loads(value), orient='index').T
        temp_df.index = [key]
        df = pd.concat([df, temp_df])
    else:
        pass

st.dataframe(df)

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [45]:
df

,quantity,unit
Chicken Breast,0,gram
Cooking Oil,496,mililitre
Sugar,-1,kilogram
Ginger Paste,498,gram
Olive Oil,498,mililitre
Turmeric powder,99,gram
Chicken,-499,kilogram
Kidney Beans,-99,kilogram
Flour,400,gram
Rice,-1,kilogram


In [46]:
def categorize(categories, grocery_data):
    # create new column
    grocery_data['category'] = ''

    # categorize
    for index, _ in grocery_data.iterrows():
        item_name = index.lower()
        for category, keywords in categories.items():
            for keyword in keywords:
                if keyword in item_name:
                    grocery_data.at[index, 'category'] = category
                    break

    return grocery_data.sort_values(by='category')

In [47]:
categorize(categories, df).head()

,quantity,unit,category
Sugar,-1,kilogram,Bakery
Durum,2,Packet,Bakery
Salt,250,gram,Bakery
Mushroom,4,Cans,Canned goods
Kidney Beans,-99,kilogram,Canned goods


In [48]:
st.dataframe(categorize(categories, df))

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [49]:
df.columns

Index(['quantity', 'unit', 'category'], dtype='object')

In [50]:
df

,quantity,unit,category
Chicken Breast,0,gram,Meat
Cooking Oil,496,mililitre,Oil
Sugar,-1,kilogram,Bakery
Ginger Paste,498,gram,Spices
Olive Oil,498,mililitre,Oil
Turmeric powder,99,gram,Spices
Chicken,-499,kilogram,Meat
Kidney Beans,-99,kilogram,Canned goods
Flour,400,gram,Grains
Rice,-1,kilogram,Grains


In [51]:
def sort_catagory(dataFrame):
    dataFrame = dataFrame.sort_values(by='category')
    return dataFrame

In [52]:
sort_catagory(df)

,quantity,unit,category
Sugar,-1,kilogram,Bakery
Durum,2,Packet,Bakery
Salt,250,gram,Bakery
Mushroom,4,Cans,Canned goods
Kidney Beans,-99,kilogram,Canned goods
Milk,2,Packets,Dairy
Butter,500,gram,Dairy
Egg,10,Piece,Dairy
Cheese,500,gram,Dairy
Tuna,150,gram,Fish


In [53]:
def unit_conversion(dataFrame):

    # Convert Kilogram to gram
    dataFrame.loc[dataFrame['unit'] == 'Kilogram', ['quantity', 'unit']] = dataFrame.loc[dataFrame['unit'] == 'Kilogram', ['quantity', 'unit']].replace({'quantity': {value: value*1000 for value in dataFrame.loc[dataFrame['unit'] == 'Kilogram', 'quantity']} , 'unit': {'Kilogram': 'gram'}})

    # # Convert Litre to millilitre
    dataFrame.loc[dataFrame['unit'] == 'Litre', ['quantity', 'unit']] = dataFrame.loc[dataFrame['unit'] == 'Litre', ['quantity', 'unit']].replace({'quantity': {value: value*1000 for value in dataFrame.loc[dataFrame['unit'] == 'Litre', 'quantity']} , 'unit': {'Litre': 'MilliLitre'}})
    
    return dataFrame

In [54]:
unit_conversion(df)

,quantity,unit,category
Chicken Breast,0,gram,Meat
Cooking Oil,496,mililitre,Oil
Sugar,-1,kilogram,Bakery
Ginger Paste,498,gram,Spices
Olive Oil,498,mililitre,Oil
Turmeric powder,99,gram,Spices
Chicken,-499,kilogram,Meat
Kidney Beans,-99,kilogram,Canned goods
Flour,400,gram,Grains
Rice,-1,kilogram,Grains


In [55]:
st.dataframe(unit_conversion(df))

DeltaGenerator(_root_container=0, _provided_cursor=None, _parent=None, _block_type=None, _form_data=None)

In [56]:
def plotting(dataFrame, index=None):
    if index is None: 
        index = dataFrame.index
    fig = px.bar(dataFrame, x=index, y='quantity')
    fig.show()
    return fig

In [57]:
plotting(df, index=None)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
import sys
print(sys.version)

In [ ]:
def plotting(dataFrame, index):

    piece_df = dataFrame.loc[dataFrame['unit'] == 'Piece']
    gram_df = dataFrame.loc[dataFrame['unit'] == 'gram']
    milliLitre_df = dataFrame.loc[dataFrame['unit'] == 'MilliLitre']

    if not piece_df.empty:
        p = piece_df.plot(kind='bar', unit=index, quantity='quantitquantity')
        p.set_xlabel('Current Items')
        p.set_ylabel('quantity (Pieces)')

    if not gram_df.empty:
        g = gram_df.plot(kind='bar', unit=index, quantity='quantity')
        g.set_xlabel('Current Items')
        g.set_ylabel('quantitquantity (gram)')

    if not milliLitre_df.empty:
        l = milliLitre_df.plot(kind='bar', unit=index, quantity='quantity')
        l.set_xlabel('Current Items')
        l.set_ylabel('quantity (Millilitre)')

    plt.show()